<a href="https://colab.research.google.com/github/zeiosis/ffnet-summary-prediction/blob/main/dangerzone/colab/f_w_thing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers==4.15.0
!pip install -U sentence-transformers==2.2.0
!pip install -U transformers[sentencepiece]
!pip install datasets

     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 59.2 MB/s 
     |████████████████████████████████| 880 kB 57.7 MB/s 
     |████████████████████████████████| 3.3 MB 52.2 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=c377b43fca44aff42540b30b703ed95cb8eec4f7d0dfbe60fd0458ba77f43129
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 79 kB 3.9 MB/s 
     |████████████████████████████████| 1.2 MB 45.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=682f7139eb655391ed524a00305e541a251b7

In [ ]:
import transformers
from transformers import pipeline
import pandas as pd

In [ ]:
classifier = pipeline(model='typeform/distilbert-base-uncased-mnli', device=0)

Downloading:   0%|          | 0.00/2.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/776 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

KeyboardInterrupt: ignored

In [ ]:
#s1 = "Sometimes the pain of others can hurt even more than one’s own. In which the reader is injured."
#s2 = "Violence isn't the only way to get what you want. There are other, more subtle ways."
#s3 = "Razor struggles to confess to Bennett with his limited vocabulary."
#s4 = "In which Xiao learns to open himself up to the world a little more after the collapse of Rex lapis's\
# contracts but it was not always easy for a soul doomed to eternal damnation. Meanwhile, Albedo liked to tempt\
#  fate where the extraordinary are concerned. If only the traveler's comrades are made of saner bunch."
#s5 = "Childe loved his family. His mother, father, Tonia, Teucer, Anthon—each and every one of them, he \
#loved to the end of the world. He also happened to be fairly head-over-heels for a funeral consultant who\
# spoke in lengthy historical prose, and acted a good six-thousand years older than he truly was. He even \
# loved the three children the consultant was caring for: two teenagers and an elementary schooler, all bright\
#  and beautiful in their own ways, just like their guardian. There was just one problem: the kids did not exactly like him back."
#s6 = "Xiao has come to two realizations. The first is that Albedo is cute, strike that he's bloody adorable, and the second is that,\
# oh no. Xiao has a type.  Or, perhaps a story of a single bright candle in a long eternity, an ever persistently kind sea that can\
#  love a stone's sharp edges without wearing them dull, only that the stone wishes to become softer, to love the sea better.\
#    Tags will be updated as they become relevant. It gets heavy from chapter two onwards to feel free to read chapter one as a standalone"
#
#summarylist = [s1, s2, s3, s4, s5, s6]

In [ ]:
k = pd.read_csv("id_tags_summary_SMALL.csv")

In [ ]:
from io import StringIO
from html.parser import HTMLParser
from re import sub

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [ ]:
pattern1 = "<\/p><p>"
fsumlist = []
for x in list(k['summary']):
    fsumlist.append(strip_tags(sub(pattern1, " ", x)))

In [ ]:
import torch
  
print(f"Is CUDA supported by this system? \
      {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
  
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device:\
      {torch.cuda.current_device()}")
        
print(f"Name of current CUDA device:\
      {torch.cuda.get_device_name(cuda_id)}")

In [ ]:
fromsum = classifier(
    fsumlist,
    candidate_labels=['Fluff', 'Hurt and Comfort', 'Angst', 'Modern AU',\
                      'Mutual Pining', 'Established Relationship', 'Light Angst',\
                      'Love Confessions', 'Pining', 'Fluff and Humor']
)

#averages 14 summaries/second, so 650 summaries >~ 46s?

In [ ]:
fheader = ['sequence']
for x in fromsum[0]['labels']:
  fheader.append(x)

fseq = pd.DataFrame(columns = fheader)

sequences = []
for x in fromsum:
  sequences.append(x['sequence'])

fseq['sequence'] = sequences

In [ ]:
k = 0

for x in fromsum:
  for y in range(len(x['labels'])):
    fseq[x['labels'][y]][k] = x['scores'][y]
  k += 1

In [ ]:
fseq.to_csv('predictions.csv')

CUT
<br>

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding, Trainer
import pandas as pd
from datasets import Dataset, DatasetDict


#raw_datasets = load_dataset("glue", "mrpc")
raw_datasets = pd.read_csv("id_tags_summary_SMALL.csv")
#raw_datasets = raw_datasets.drop('Unnamed: 0', axis=1)
raw_datasets = raw_datasets.rename(columns={'summary': 'text', 'Unnamed: 0': 'ID'})

raw_datasets = raw_datasets.drop(['Unnamed: 0.1', 'work_id'], axis = 1)
raw_datasets = Dataset.from_dict(raw_datasets)

# 90% train, 10% test + validation
ttv_ds = raw_datasets.train_test_split(test_size=0.1)
# Split the 10% test + valid in half test, half valid
test_valid = ttv_ds['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
raw_datasets = DatasetDict({
    'train': ttv_ds['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

#checkpoint = "bert-base-uncased"
checkpoint = "typeform/distilbert-base-uncased-mnli"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


#def tokenize_function(example):
#    return tokenizer(example["text"], truncation=True)


#tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
raw_datasets

In [ ]:
example = raw_datasets['train']
example[0]

In [ ]:
labels = [label for label in raw_datasets['train'].features.keys() if label not in ['ID', 'text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [ ]:
encoded_dataset = raw_datasets.map(preprocess_data, batched=True, remove_columns=raw_datasets['train'].column_names)

In [ ]:
example = encoded_dataset['train'][0]
print(example.keys())

In [ ]:
tokenizer.decode(example['input_ids'])

In [ ]:
example['labels']

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

In [ ]:
encoded_dataset.set_format("torch")

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

In [ ]:
batch_size = 8
metric_name = "f1"

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [ ]:
encoded_dataset['train'][0]['labels'].type()

In [ ]:
encoded_dataset['train']['input_ids'][0]

In [ ]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

# PLS IGNORE

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer-model-and-checkpoints")

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3) #why 3????

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
predictions = trainer.predict(tokenized_datasets["valid"])
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", problem_type="multi_label_classification")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]
